In [0]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import Lambda

Using TensorFlow backend.


In [0]:
# load doc into memory
def load_doc(filename):
	file = open(filename, 'r', encoding='utf-8')
	text = file.read()
	file.close()
	return text

In [0]:
shakeraw_text = load_doc('shakespeare.txt')
shakeraw_text = shakeraw_text.lower()

spenraw_text = load_doc('spenser.txt')
spenraw_text = spenraw_text.lower()

raw_text = shakeraw_text + spenraw_text

In [0]:
# map letters to numbers and numbers to letters
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
# set sequence length to 40
# dataX is input with len 40 sequences
# dataY is next letter in text file
seq_length = 40
n_chars = len(raw_text)
dataX = []
dataY = []

# iterate text file with step size 3
for i in range(0, n_chars - seq_length, 3):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])


vocab_size = len(char_to_int)

# one hot encode input variables
sequences = [to_categorical(x, num_classes=vocab_size) for x in dataX]
X = np.array(sequences)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# model with 200 units

model = Sequential()
model.add(LSTM(200,input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])

# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=100, batch_size=64, verbose=2, shuffle=True, callbacks=callbacks_list)
scores = model.evaluate(X,y,verbose=1,batch_size=20)
print('Accurracy: {}'.format(scores[1])) 






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





 - 52s - loss: 2.5913 - acc: 0.2644

Epoch 00001: loss improved from inf to 2.59132, saving model to weights.best.hdf5
Epoch 2/100
 - 48s - loss: 2.1277 - acc: 0.3755

Epoch 00002: loss improved from 2.59132 to 2.12771, saving model to weights.best.hdf5
Epoch 3/100
 - 48s - loss: 1.9649 - acc: 0.4145

Epoch 00003: loss improved from 2.12771 to 1.96495, saving model to weights.best.hdf5
Epoch 4/100
 - 49s - loss: 1.8601 - acc: 0.4416

Epoch 00004: loss improved from 1.96495 to 1.86013, saving model to weights.best.hdf5
Epoch 5/100
 - 48s - loss: 1.7824 - acc: 0.4607

Epoch 00005: loss improved from 1.86013 to 1.78240, saving model to weights.best.hdf5
Epoch 6/100
 - 49s - loss: 1.7176 - acc: 0.4767

Epoch 00006: loss improved from 1.78240 to 1.71756, saving model to weights.best.hdf5
Epoch 7/100
 - 48s - loss: 1.6573 - acc: 0.4915

Epoch 00007: loss improved from 1.7175

In [0]:
def sample(preds, temp):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds[0], 1)
    return np.argmax(probas)

In [0]:
# initial 40 character seed
seed = "shall i compare thee to a summer's day?\n"
for temperature in [0.25, 0.75, 1.5]:
  print("\n new temperature \n")
  # load best weights
  model = Sequential()
  model.add(LSTM(200,input_shape=(X.shape[1], X.shape[2])))
  model.add(Dense(y.shape[1],activation='softmax'))
  model.load_weights("weights.best.hdf5")
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  pattern = [char_to_int[value] for value in seed]
  sys.stdout.write(seed)

  # generate characters
  for i in range(700):
    sequences = [to_categorical(x, num_classes=vocab_size) for x in pattern]
    x = np.array(sequences)
    x = x.reshape(1, x.shape[0], x.shape[1])
    prediction = model.predict(x, verbose=0)
    index = sample(prediction, temperature)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


 new temperature 

shall i compare thee to a summer's day?
when thou wilt both now inch me a sail,
which with upless and parain from her dight,
the winger beauty glornoty simmed flime,
whose sulf to be doth being hadd not fon:
when in his must enoust fioth heaven asporn,
(ar if the such dost parter, which brignt
in dart love betught thou did chows dird-and his dreast to be.



thou art he sermint of your praises to be,
sieden sush abord mad beine are not thee.



thy heart love your me ause of wist,
so groding the with of mer's light night,
and dark from have sweet with thee wist,
is but thou braint the sin every will.



why have your cold distirity,
verpery can mine with self so stornate,
then in her praise and chenuly stound,
that in her will see
 new temperature 

shall i compare thee to a summer's day?
when thou well bund his chuse and his pleasure,
of the fire of geader prece that true how storl:
and wist in heaven ye dy mus cannet pire,
the chreppece of thy sour feauty with bea